In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
pip install underthesea

     |████████████████████████████████| 7.6 MB 5.3 MB/s 
     |████████████████████████████████| 965 kB 44.2 MB/s 
     |████████████████████████████████| 235 kB 47.3 MB/s 
     |████████████████████████████████| 581 kB 53.5 MB/s 


In [213]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from underthesea import sent_tokenize,word_tokenize
from itertools import chain
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.metrics import *
from sklearn.utils.class_weight import compute_class_weight
import string
import re
from os.path import join
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [5]:
root_data="/content/drive/MyDrive/vietnamese_students_feedback/preproccesd_data"

In [6]:
train=pd.read_csv(f'{root_data}/train.csv')

In [7]:
train

,sentence,sentiment,topic,cleaned_sentence,seq_len
0,giọng cô nghe dễ ngủ !,1,0,giọng cô nghe dễ ngủ,5
1,phòng thực hành nhỏ không đủ máy cho sinh viên...,0,2,phòng thực hành nhỏ không đủ máy cho sinh viên...,14
2,nội dung học cần cải thiện .,0,1,nội dung học cần cải thiện,6
3,làm nhiều bài kiểm tra quá .,0,1,làm nhiều bài kiểm tra quá,6
4,phần thi nói giữa các lớp không thống nhất về ...,0,1,phần thi nói giữa các lớp không thống nhất về ...,15
...,...,...,...,...,...
11421,cô đưa ra nhiều hướng giải quyết khi sinh viên...,2,0,cô đưa ra nhiều hướng giải quyết khi sinh viên...,16
11422,em nghĩ nội dung thực hành nên gần với đồ án h...,0,1,em nghĩ nội dung thực hành nên gần với đồ án hơn,12
11423,luôn nhiệt tình và tận tâm .,2,0,luôn nhiệt tình và tận tâm,6
11424,nó rất rập khuôn và không thể khiến cho sinh v...,0,1,nó rất rập khuôn và không thể khiến cho sinh v...,15


In [23]:
valid=pd.read_csv(f'{root_data}/valid.csv')

In [17]:
def tokenizer(s): 
    sentences = sent_tokenize(s) 
    # tokens=[]
    _sentences=[]
    for sentence in sentences:
      # tokens+=sentence
      tokens=[]
      for token in word_tokenize(sentence,):
        token=token.replace("-", " ")
        token=token.replace(" ","_")
        
        tokens.append(token)
      _sentences.append(tokens)
    return list(chain.from_iterable(_sentences))

In [18]:
tokenizer("làm nhiều bài kiểm tra quá .")

['làm', 'nhiều', 'bài', 'kiểm_tra', 'quá', '.']

In [19]:
train_x=train['cleaned_sentence'].tolist()

In [24]:
valid_x=valid['cleaned_sentence'].tolist()

In [20]:
train_y=train['topic'].tolist()

In [25]:
valid_y=valid['topic'].tolist()

#Convert text to count vectorizer

In [124]:
count_vectorizer=CountVectorizer(tokenizer=tokenizer,ngram_range=(1,3),max_features=5000)

In [125]:
train_vectorizer=count_vectorizer.fit_transform(train_x)

In [132]:
vocabs=count_vectorizer.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [135]:
len(vocabs)

5000

In [136]:
vocab2idx={vocab:idx for idx,vocab in enumerate(vocabs)}

In [126]:
# class_weight=compute_class_weight(y=train_y,classes=[0,1,2,3],class_weight='balanced')
# class_weight

In [ ]:
def create_inputs(sample):
  tokens=tokenizer(sample)
  size_samples=len(tokens)
  vectors=np.zeros((1,len(vocabs)))
  
  #unigram
  for token in tokens:
    if token in vocab2idx:
      vectors[:,vocab2idx[token]]+=1

  #bigram
  for idx in range(size_samples-1):
    token=" ".join(tokens[idx:idx+2])
    if token in vocab2idx:
      vectors[:,vocab2idx[token]]+=1

  #trigram
  for idx in range(size_samples-2):
    token=" ".join(tokens[idx:idx+3])
    if token in vocab2idx:
      vectors[:,vocab2idx[token]]+=1

  return vectors



In [ ]:
valid_vectorizer=[create_inputs(sample) for sample in valid_x]
valid_vectorizer=np.concatenate(valid_vectorizer,axis=0)

In [ ]:
test=pd.read_csv('/content/drive/MyDrive/vietnamese_students_feedback/raw_dataset/test.csv')

In [ ]:
punctuations=list(string.punctuation)
def normalizer(text,joinf=False):
  def remove_symbols(s):
    return re.sub(r'\b(colon\w+)\b'," ",s)

  def remove_fraction(s):
    return re.sub(r'\b(fraction)\b','/',s)

  def replace_dot(s):
    return s.replace('dot','.')
    # return re.sub(r'\b(dot)\b',".",s)

  def replace_doubledot(s):
    return s.replace('doubledot',':')
    
  def uncased(s):
    return s.lower()

  def fix_whitespaces(s):
    return re.sub(r'\s+',' ',s)

  def remove_entities(s):
    return re.sub(r'\b((\w+|)wzjwz\d+)\b'," ",s)

  def remove_vv(s):
    return s.replace('v.v'," ")

  def remove_punctuations(s):
      return ''.join(ch for ch in s if ch not in punctuations)
  def norm_major(s):
      majors={
          'cnpm':"công nghệ phần mềm",
          'khmt':"khoa học máy tính",
          'ktmt':"kĩ thuật máy tính",
          'cntt':"công nghệ thông tin",
          'av1':"anh văn 1",
          'av2':"anh văn 2",
          'av3':"anh văn 3"
      }
      for k,v in majors.items():
        s=re.sub(r'\b({})\b'.format(k),v,s)
      return s
  def remove_ps(s):
    return s.replace("p / s"," ")

  def tokenize(s,join=False): 
    sentences = sent_tokenize(s) 
    # tokens=[]
    _sentences=[]
    for sentence in sentences:
      # tokens+=sentence
      tokens=[]
      for token in word_tokenize(sentence,):
        token=token.replace("-", " ")
        if join==True:
          token=token.replace(" ","_")
        
        tokens.append(token)
      _sentences.append(" ".join(tokens))
    return " ".join(_sentences)
  
  return fix_whitespaces(uncased(remove_punctuations(tokenize(norm_major(remove_ps(remove_fraction(remove_vv(remove_entities(remove_symbols(replace_dot(replace_doubledot(text)))))))),join)))).strip()

In [ ]:
test_x=test['sentence'].map(normalizer).tolist()
test_y=test['topic'].tolist()

In [ ]:
test_vectorizer=[create_inputs(sample) for sample in test_x]
test_vectorizer=np.concatenate(test_vectorizer,axis=0)

#Decision tree

In [195]:
dt_clf=DecisionTreeClassifier(class_weight='balanced')

In [203]:
param_grid={'criterion':["gini", "entropy"],'max_depth':np.arange(10,100,10)}
dt_cv=GridSearchCV(estimator=dt_clf,param_grid=param_grid,cv=5,scoring='f1_weighted')

In [204]:
dt_cv.fit(train_vectorizer,train_y)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(class_weight='balanced'),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': array([10, 20, 30, 40, 50, 60, 70, 80, 90])},
             scoring='f1_weighted')

In [201]:
dt_cv.best_params_

{'criterion': 'entropy', 'max_depth': 90}

In [206]:
dt_cv.best_score_

0.7979845749943472

#Random Forest

In [214]:
rf_clf=RandomForestClassifier(class_weight='balanced')

In [215]:
param_grid={'n_estimators':np.arange(10,100,10),'criterion':["gini", "entropy"],'max_depth':np.arange(10,100,10)}
rf_cv=GridSearchCV(estimator=rf_clf,param_grid=param_grid,cv=5,scoring='f1_weighted')

In [216]:
rf_cv.fit(train_vectorizer,train_y)

GridSearchCV(cv=5, estimator=RandomForestClassifier(class_weight='balanced'),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': array([10, 20, 30, 40, 50, 60, 70, 80, 90]),
                         'n_estimators': array([10, 20, 30, 40, 50, 60, 70, 80, 90])},
             scoring='f1_weighted')

In [217]:
rf_cv.best_params_

{'criterion': 'gini', 'max_depth': 60, 'n_estimators': 70}

In [218]:
rf_cv.best_score_

0.8401182071804655

# K-fold SVM

In [127]:
svm_clf=SVC(class_weight='balanced')

In [128]:
param_grid={'C':[1e-5,1e-3,1e-1,1,10,100]}
svm_cv=GridSearchCV(estimator=svm_clf,param_grid=param_grid,cv=5,scoring='f1_weighted')

In [129]:
svm_cv.fit(train_vectorizer,train_y)

GridSearchCV(cv=5, estimator=SVC(class_weight='balanced'),
             param_grid={'C': [1e-05, 0.001, 0.1, 1, 10, 100]},
             scoring='f1_weighted')

In [130]:
svm_cv.best_params_

{'C': 10}

In [131]:
svm_cv.best_score_

0.8355534601177199

#Classifier on valid set

In [219]:
# valid_pred=svm_cv.predict(valid_vectorizer).tolist()
# valid_pred=dt_cv.predict(valid_vectorizer).tolist()
valid_pred=rf_cv.predict(valid_vectorizer).tolist()

In [220]:
confusion_matrix(valid_pred,valid_y)

array([[1021,   45,    3,   22],
       [  55,  191,    6,   12],
       [   0,    3,   58,    3],
       [  75,   28,    3,   58]])

In [221]:
print(classification_report(valid_pred,valid_y,digits=5))

              precision    recall  f1-score   support

           0    0.88705   0.93584   0.91079      1091
           1    0.71536   0.72348   0.71940       264
           2    0.82857   0.90625   0.86567        64
           3    0.61053   0.35366   0.44788       164

    accuracy                        0.83891      1583
   macro avg    0.76038   0.72981   0.73593      1583
weighted avg    0.82741   0.83891   0.82909      1583



#Classifier on test set

In [222]:
# test_pred=svm_cv.predict(test_vectorizer).tolist()
# test_pred=dt_cv.predict(test_vectorizer).tolist()
test_pred=rf_cv.predict(test_vectorizer).tolist()

In [223]:
confusion_matrix(test_pred,test_y)

array([[2005,   89,    6,   30],
       [ 128,  419,   10,   33],
       [  10,    5,  123,    2],
       [ 147,   59,    6,   94]])

In [224]:
print(classification_report(test_pred,test_y,digits=5))

              precision    recall  f1-score   support

           0    0.87555   0.94131   0.90724      2130
           1    0.73252   0.71017   0.72117       590
           2    0.84828   0.87857   0.86316       140
           3    0.59119   0.30719   0.40430       306

    accuracy                        0.83418      3166
   macro avg    0.76188   0.70931   0.72397      3166
weighted avg    0.82020   0.83418   0.82201      3166

